In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import pandas_datareader as pdr
import time

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [272]:
# 백테스트 날짜, 초기 잔고, 리밸런싱 주기
start = datetime(2000, 1, 1)
end = datetime(2020, 12, 31)
init_balance = 1000

rebalancing_months = [12 , 3, 10] # 리밸런싱 실행 월
rebalancing_target = -1 # -1 월 말, 0 월 초
rebalancing_dates = [-2, -1, 0, 1, 2] # 월말 3영업일, 월초 2영업일

fee = 0.5 # 거래비용 및 슬리피지. %(퍼센트) 단위

# port = []
port = {
        'asset': ['251350.KS', '195980.KS', '304660.KS', '305080.KS', '132030.KS'],
        'ratio': [42, 18, 18, 12, 10],
        'name': 'Portfolio 1'
}

In [3]:
# def individual_port_balance(port): # 개별 포트폴리오에 대한 잔액 계산
    
# 각 종목 별 수정 주가 데이터를 크롤링 해 리스트로 시계열 데이터 시리즈를 값으로 갖는 리스트 생성
price_list = [pdr.get_data_yahoo(asset, start, end)['Adj Close'] for asset in port['asset']]

df_list = pd.concat(price_list, axis=1) #주가 데이터 리스트를 데이터프레임으로 변환
df_list.columns = port['asset']#[f'{asset}'.strip('.KS') for asset in port['asset']] # 포트를 구성하는 종목 명을 컬럼 명으로
first_day = df_list.dropna().index.min() # 전 종목 결측치 없는 최초 일자를 거래시작일로 지정
df_price = df_list[first_day:].fillna(method='pad') # 최초일 이후의 결측치는 전일종가로 채움
end_day = df_price.index.max()

## 코드 튜닝

In [4]:
df_price.head(3)

,251350.KS,195980.KS,304660.KS,305080.KS,132030.KS
Date,,,,,
2018-09-10,13050.0,10085.0,9996.469727,10140.0,8780.0
2018-09-11,13115.0,10140.0,9995.849609,10095.0,8805.0
2018-09-12,13115.0,10075.0,9960.000000,10100.0,8780.0


### 리밸런싱 날짜 계산

In [5]:
df_price['Date'] = df_price.index
#df_price['year'] = df_price.index.year
df_price['month'] = df_price.index.month
df_price['year_month'] = df_price.index.strftime('%Y-%m')
# annual = df_price.groupby('year').apply(lambda x: x.iloc[[-1]]).reset_index('Date').drop(['year_month', 'year', 'Date'], axis=1)
# monthly = df_price.groupby('year_month').apply(lambda x: x.iloc[[-1]]).reset_index('Date').drop(['year_month', 'year', 'Date'], axis=1)
#rebalancing_days = df_price.groupby('month').apply(lambda x: x.iloc[[-1]])

# for port in port_list:
#     for i in (range(len(annual)-1, 0, -1)):
#         annual[port][i] = round((annual[port][i] / annual[port][i-1] - 1), 4) * 100
#     for i in (range(len(monthly)-1, 0, -1)):
#         monthly[port][i] = round((monthly[port][i] / monthly[port][i-1] - 1), 4) * 100
# annual.iloc[0] = (annual.iloc[0] / init_balance - 1) * 100
# monthly.iloc[0] = (monthly.iloc[0] / init_balance - 1) * 100

In [6]:
df_price

,251350.KS,195980.KS,304660.KS,305080.KS,132030.KS,Date,month,year_month
Date,,,,,,,,
2018-09-10,13050.0,10085.0,9996.469727,10140.0,8780.0,2018-09-10,9,2018-09
2018-09-11,13115.0,10140.0,9995.849609,10095.0,8805.0,2018-09-11,9,2018-09
2018-09-12,13115.0,10075.0,9960.000000,10100.0,8780.0,2018-09-12,9,2018-09
2018-09-13,13160.0,10165.0,9970.000000,10050.0,8850.0,2018-09-13,9,2018-09
2018-09-14,13070.0,10290.0,9975.000000,10010.0,8860.0,2018-09-14,9,2018-09
...,...,...,...,...,...,...,...,...
2020-09-29,15290.0,10440.0,13840.000000,12510.0,12995.0,2020-09-29,9,2020-09
2020-10-05,15295.0,10540.0,13625.000000,12420.0,13070.0,2020-10-05,10,2020-10
2020-10-06,15375.0,10630.0,13405.000000,12330.0,13190.0,2020-10-06,10,2020-10


In [7]:
df_target = df_price.loc[df_price["month"].isin(rebalancing_months)].groupby('year_month').apply(lambda x: x.iloc[rebalancing_target])
df_target

,251350.KS,195980.KS,304660.KS,305080.KS,132030.KS,Date,month,year_month
year_month,,,,,,,,
2018-10,12320.0,9450.0,9530.0,10160.0,8945.0,2018-10-31,10,2018-10
2018-12,11300.0,9600.0,10225.0,10195.0,9350.0,2018-12-28,12,2018-12
2019-03,12960.0,10520.0,10575.0,10645.0,9350.0,2019-03-29,3,2019-03
2019-10,14355.0,10530.0,11720.0,11410.0,10730.0,2019-10-31,10,2019-10
2019-12,15160.0,11175.0,11555.0,11275.0,10810.0,2019-12-30,12,2019-12
2020-03,12475.0,8300.0,14020.0,12905.0,11545.0,2020-03-31,3,2020-03
2020-10,15425.0,10720.0,13435.0,12260.0,13035.0,2020-10-08,10,2020-10


In [274]:
rebal_list = list(df_target['Date'])

### 리밸런싱 분산용 코드 검토

In [277]:
if rebal_list[0] != first_day:
    rebal_list.insert(0, first_day)
if rebal_list[-1] != end_day:
    rebal_list.append(end_day)
rebal_list

[Timestamp('2018-09-10 00:00:00'),
 Timestamp('2018-10-31 00:00:00'),
 Timestamp('2018-12-28 00:00:00'),
 Timestamp('2019-03-29 00:00:00'),
 Timestamp('2019-10-31 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2020-03-31 00:00:00'),
 Timestamp('2020-10-08 00:00:00')]

In [295]:
rebal_dates_list = []
for target in range(len(rebal_list)):
    for date in rebalancing_dates:
        if target == 0:
            if date >= 0:
                rebal_dates_list.append(rebal_list[target] + timedelta(days = date))
        elif target == len(rebal_list)-1:
            if date <= 0:
                rebal_dates_list.append(rebal_list[target] + timedelta(days = date))
        else:
            rebal_dates_list.append(rebal_list[target] + timedelta(days = date))
rebal_dates_list

[Timestamp('2018-09-10 00:00:00'),
 Timestamp('2018-09-11 00:00:00'),
 Timestamp('2018-09-12 00:00:00'),
 Timestamp('2018-10-29 00:00:00'),
 Timestamp('2018-10-30 00:00:00'),
 Timestamp('2018-10-31 00:00:00'),
 Timestamp('2018-11-01 00:00:00'),
 Timestamp('2018-11-02 00:00:00'),
 Timestamp('2018-12-26 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-28 00:00:00'),
 Timestamp('2018-12-29 00:00:00'),
 Timestamp('2018-12-30 00:00:00'),
 Timestamp('2019-03-27 00:00:00'),
 Timestamp('2019-03-28 00:00:00'),
 Timestamp('2019-03-29 00:00:00'),
 Timestamp('2019-03-30 00:00:00'),
 Timestamp('2019-03-31 00:00:00'),
 Timestamp('2019-10-29 00:00:00'),
 Timestamp('2019-10-30 00:00:00'),
 Timestamp('2019-10-31 00:00:00'),
 Timestamp('2019-11-01 00:00:00'),
 Timestamp('2019-11-02 00:00:00'),
 Timestamp('2019-12-28 00:00:00'),
 Timestamp('2019-12-29 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2019-12-31 00:00:00'),
 Timestamp('2020-01-01 00:00:00'),
 Timestamp('2020-03-

### 리밸런싱 진행 후 잔액 계산

In [265]:
df_ratio = df_price.drop(['year_month', 'month', 'Date'], axis=1)
tmp_balance = init_balance
new_col = [f'{asset}_@' for asset in port['asset']]

In [ ]:
for i in range(len(rebal_list)-1):
    tmp = df_ratio.loc[(df_ratio.index >= rebal_list[i]) & (df_ratio.index <= rebal_list[i+1])]
    for asset, ratio in zip(port['asset'], port['ratio']):
        changing_val = tmp[asset] / tmp[asset].iloc[0] * ratio/sum(port['ratio']) * tmp_balance
        df_ratio.loc[(df_ratio.index >= rebal_list[i]) & (df_ratio.index <= rebal_list[i+1]), f'{asset}_@'] \
        = changing_val
    tmp_balance = df_ratio.loc[rebal_list[i+1], new_col].sum()

=====================================================================

In [ ]:
df_ratio['Balance'] = df_ratio[new_col].sum(axis=1)

In [265]:
# df_ratio = df_price.drop(['year_month', 'month', 'Date'], axis=1)
# tmp_balance = init_balance
# new_col = [f'{asset}_@' for asset in port['asset']]
# for i in range(len(rebal_list)-1):
#     tmp = df_ratio.loc[(df_ratio.index >= rebal_list[i]) & (df_ratio.index <= rebal_list[i+1])]
#     for asset, ratio in zip(port['asset'], port['ratio']):
#         changing_val = tmp[asset] / tmp[asset].iloc[0] * ratio/sum(port['ratio']) * tmp_balance
#         df_ratio.loc[(df_ratio.index >= rebal_list[i]) & (df_ratio.index <= rebal_list[i+1]), f'{asset}_@'] \
#         = changing_val
#     tmp_balance = df_ratio.loc[rebal_list[i+1], new_col].sum()
# #     print(tmp_balance)
# #     print(df_ratio.loc[rebal_list[i+1]])
# #     print('================================')
# df_ratio['Balance'] = df_ratio[new_col].sum(axis=1)

In [266]:
#df_ratio.to_excel('df_ratio.xlsx')